In [13]:
#imports and reading in files + path setup
import os
import pandas as pd
flatfiles=[]
for filename in os.listdir("genomes"):
    flatfiles.append(filename)

path='C:\\Users\\Matt\\Desktop\\Bioinfo Project\\genomes\\'

In [14]:
def parseFile2(filename,path): 
    contents=open(path+filename).read()
    #Opens fasta file
#    path = "C:\\Users\\camos333\\Desktop\\AF046869.txt"
#    f=open(path, "r")
#    if f.mode == "r":
#        contents=f.read()
    
    #Imports Regular Expressions
    import re
    #import pandas as pd\
    
    #Finds the character number that the word ORIGIN ends and the nucleic acid sequence begins
    def nucleostartno(dna):
        startno = re.search(r'\nORIGIN', dna).end()
        return startno
    
    #Trims the sequence to remove spaces, numbers, slashes, and replaces t to u respectively
    def justactg1234ss(dna):
        nucleo = re.sub(r'([\s])','',dna)
        return nucleo
    def justactgss(dna):
        nucleo = re.sub(r'\d','',dna)
        return nucleo
    def justactg(dna):
        nucleo = re.sub(r'/','',dna)
        return nucleo
    def justacug(dna):
        nucleo = re.sub(r't','u',dna)
        return nucleo
    #Sets contents to trim itself to everything past the word ORIGIN
    cutoffORIGIN = str(contents[((nucleostartno(contents))+7):])
    
    #Goes through and uses those 4 functions
    actgstring1 = justactg1234ss(cutoffORIGIN)
    actgstring2 = justactgss(actgstring1)
    actgstring3 = justactg(actgstring2)
    actgstring = str.upper(justacug(actgstring3))
    #quick capitalization
    
    #actgstring is the isolated nucleotide sequence from the GenBank file.
    
    
    
    
    '''FROME HERE ON IS WHERE IT ISOLATES FROM actgstring WHERE EACH CDS IS'''
    
    #Function that splits Genbank File by where the word 'CDS' is
    def CDSsplit1(dna):
        separate = re.split(r'CDS', dna)
        return separate
    
    x = (CDSsplit1(contents))
    
    #Cuts off everything in the GenBank file thats before the first 'CDS'
    x.pop(0)
    
    #Once CDS is found, these will select and trim the range to produce the starting number, ensuring there are not carrots.
    def CDSstartnucleo1(dna):
        startno = ''.join(re.findall(r'\s\d*[.]{2}',dna))
        return startno
    def CDSstartnucleo2(dna):
        startno = re.sub(r'[.]','',dna)
        return startno
    
    #Once CDS is found, these will select and trim the range to produce the ending number
    def CDSendnucleo1(dna):
        startno = ''.join(re.findall(r'[.]{2}\d*',dna))
        return startno
    def CDSendnucleo2(dna):
        startno = re.sub(r'[.]','',dna)
        return startno
    
    def augfinder(dna):
        startno = re.search(r'AUG',dna).start()
        return startno
    '''EDIT NEXT LINE FOR MASS GENERIC FILE MANIPULATION'''
    #Creates the file (if it does not exist) 'CleanedCDS.txt'. I don't know how you'd want to do the file manipulation, so in order for you to execute this file in mass you might want to change how the name thing works

    
    #Prelimenary lists
    truestartlist = []
    falsestartlist = []
    
    #Big Boi lists
    trueStarts = []
    falseStarts = []
    
    #This for loop takes the split GenBank file and isolates the correct sequences from the main 'actgstring' string using the numbers from the CDS start and end numbers
    for i in x:
    #Trims 13 characters up from 'CDS', which is exactly when the numbers start that reference where they are in the genome. 30 characters up is where the numbers have certainly ended. It's fine if the numbers are not exact
        i = i[10:30]
    #Ensures the those carrot CDS's are not included
        if re.search(r'\s\d*[.]{2}',i) is not None:
    #Isolates start number and end number using the functions created earlier
            startno = int(CDSstartnucleo2(CDSstartnucleo1(i)))
    #            endno = int(CDSendnucleo2(CDSendnucleo1(i)))
    #References 'actgstring' and trims it to only include the coding range for it's respective CDS.
            seqt = actgstring[(startno - 10):(startno + 8)]
            truestartno = startno-1
            truestartlist.append(truestartno)
    #LAYOUT. IF YOU THINK ANOTHER LAYOUT WOULD BE MORE USEFUL, THIS IS WHERE YOU NEED TO CHANGE IT. Prepares the layout of text_file. 'entry1' includes 'CDS ###..###' and 'entry2' (after a newline \n) holds the actual coding region. I made 3 newlines in between each CDS.
            trueStarts.append(seqt)
    #puts true starts into the correct spot in the dictionary startsoutput
            
    
    
    #for use in loop below
    location = -1
    
    # Loop while true, finds AUGs in raw actg sequence and checks if they are in the true list, if so, they are deleted.
    while True:
        # Advance location by 1.
        location = actgstring.find("AUG", location + 1)
        
        if location == -1: break
    
    
        if not location in truestartlist: falsestartlist.append(location)
    
    
    
    #constructs falseStarts
    for n in falsestartlist:
        falseStarts.append(actgstring[(n - 9):(n + 9)])
    
    
    
    #constructs final dictionary
    Final = {"trueStarts":trueStarts,"falseStarts":falseStarts}
    #print (Final)
    return Final

In [15]:
def parseFile(filename, path):
    fileString=open(path+filename).read()
    
    #AQUIRE START INDICIES
    startIndex=fileString.find("CDS")
    starts=[]
    while startIndex>0:
        temp=fileString[startIndex+16:fileString.find(".",startIndex)]
        if temp[0]!='>' and temp[0]!='<':
            starts.append(int(temp))
            #print(filename)
        startIndex=fileString.find("CDS",startIndex+1)
    #print(starts)
    
    #ISOLATE GENOME
    genome=""
    for char in fileString[fileString.find("ORIGIN"):len(fileString)]:
        if char=='t'or char=='g' or char=='a' or char=='c':
            genome+=char
    
    #CHANGE T BASE TO U TO REPRESENT AS RNA
    genomeRNA=""
    for char in genome:
        if char=='t':
            genomeRNA+='u'
        else:
            genomeRNA+=char
    
    #Get all start codons in genome
    startCodonIndicies=[]
    for i in range(10,len(genomeRNA)-10):
        #print(genome[i:i+3])
        if genomeRNA[i:i+3]=='aug':
            startCodonIndicies.append(i+1)
    
    falsePositiveStarts=[]
    for index in startCodonIndicies:
        if index in starts:
            pass
        else:
            falsePositiveStarts.append(index)
    
    trueStarts=[]
    for index in starts:
        trueStarts.append(genomeRNA[index-10:index+8]+filename)
    falseStarts=[]
    for index in falsePositiveStarts:
        falseStarts.append(genomeRNA[index-10:index+8]+filename)
    
    return {"trueStarts": trueStarts, "falseStarts": falseStarts}

In [16]:
#develop scoring matrix


#load true and false start spaces
trueStartSpace=[]
falseStartSpace=[]
#first 10 of flat files
for i in range(100):
    spaces=parseFile(flatfiles[i],path)
    trueStartSpace+=spaces["trueStarts"]
    falseStartSpace+=spaces["falseStarts"]

numNucs=18
#build consensus matrices
print("True start codon identities by position")
startIdentities={"A": [0 for i in range(numNucs)], "U": [0 for i in range(numNucs)], "G": [0 for i in range(numNucs)], "C": [0 for i in range(numNucs)]}
for i in trueStartSpace:
    temp=i.replace(" ","").upper()
    for n in range(numNucs):
        try:
            startIdentities[temp[n]][n]+=1
        except:
            print("except: " + temp)
#make data frame of start identities for clean format of output
startIDsDF=pd.DataFrame.from_dict(startIdentities)
print(startIDsDF.T)

print("\nTrue start codon frequency by position")
for key in startIdentities:
    for i in range(numNucs):
        startIdentities[key][i]=round(startIdentities[key][i]/len(trueStartSpace),2)
startFreqsDF=pd.DataFrame.from_dict(startIdentities)
print(startFreqsDF.T)

#print("\nFalse start codon identities by position")
falseStartIdentities={"A": [0 for i in range(numNucs)], "U": [0 for i in range(numNucs)], "G": [0 for i in range(numNucs)], "C": [0 for i in range(numNucs)]}
for i in falseStartSpace:
    temp=i.replace(" ","").upper()
    for n in range(numNucs):
        falseStartIdentities[temp[n]][n]+=1
#make data frame of false start identities for clean format of output
falseStartIDsDF=pd.DataFrame.from_dict(falseStartIdentities)
#print(falseStartIDsDF.T)

#print("\nFalse start codon frequency by position")
for key in falseStartIdentities:
    for i in range(numNucs):
        falseStartIdentities[key][i]=round(falseStartIdentities[key][i]/len(falseStartSpace),2)
falseStartFreqsDF=pd.DataFrame.from_dict(falseStartIdentities)
#print(falseStartFreqsDF.T)

True start codon identities by position
    0    1    2    3    4    5    6    7    8    9    10   11   12   13   14  \
A   98    0  204  107   34  487  420  203  403  717    0    0  100  102  208   
C  122  204  108  104  280    9    0  305  100    0    0    0   99  371   22   
G    9  307   99  113  115   10  297  200    9    0    0  717  329  128  189   
U  488  206  306  393  288  211    0    9  205    0  717    0  189  116  298   

    15   16   17  
A  185  207    8  
C    8  210   27  
G  284  299  468  
U  240    1  214  

True start codon frequency by position
     0     1     2     3     4     5     6     7     8    9    10   11    12  \
A  0.14  0.00  0.28  0.15  0.05  0.68  0.59  0.28  0.56  1.0  0.0  0.0  0.14   
C  0.17  0.28  0.15  0.15  0.39  0.01  0.00  0.43  0.14  0.0  0.0  0.0  0.14   
G  0.01  0.43  0.14  0.16  0.16  0.01  0.41  0.28  0.01  0.0  0.0  1.0  0.46   
U  0.68  0.29  0.43  0.55  0.40  0.29  0.00  0.01  0.29  0.0  1.0  0.0  0.26   

     13    14    15    

In [17]:
matrix=startFreqsDF
#score a sequence based on scoreing matrix
def scoreSequence(seq, matrix):
    seq=seq.upper()
    score=1
    for i in range(numNucs):
        positionScore=matrix[seq[i]][i]
        if positionScore==0:
            score*=.000001
        else:
            score*=positionScore
    return score

In [19]:
#true start scores
print("True start Scores")
trueLow=99999999
trueHigh=0
falseLow=999999
falseHigh=0
for i in range(len(trueStartSpace)):
    s=scoreSequence(trueStartSpace[i],matrix)
    if s<trueLow:
        trueLow=s
    if s>trueHigh:
        trueHigh=s
    if s<.00000001:
        print(trueStartSpace[i])
    #print(s)
#false start scores
print("False Start Scores:")
for i in range(len(falseStartSpace)):
    s=scoreSequence(falseStartSpace[i],matrix)
    if s<falseLow:
        falseLow=s
    if s>falseHigh:
        falseHigh=s
    #print(s)

print("True high: " + str(trueHigh))
print("True low: " + str(trueLow))
print("False high: " + str(falseHigh))
print("False Low: " + str(falseLow))

True start Scores
ugaaaugaaauggggucuAF494042.txt
ugaaaugaaauggggucuAY150312.txt
ugaaaugaaauggggucuAY262352.txt
uuuggaguaaugcaauggAY366525.txt
aguaaugcaauggggucaAY366525.txt
cuucagacaauggcucauAY366525.txt
cuuggcgauaugagauguAY366525.txt
ggcugagcaaugggaggcAY366525.txt
ugaaaugaaaugggguccAY424271.txt
ugaaaugaaaugggguccAY585241.txt
uucggggugaugcaauggAY588319.txt
ggugaugcaauggggucaAY588319.txt
uuucagacaauggcucauAY588319.txt
cuuggcaauaugagauguAY588319.txt
ggcugagcaaugggaggcAY588319.txt
ugaaaugaaaugggguccAY612613.txt
ugaaaugaaaugggguccDQ473474.txt
agcggaacaauggggucaDQ473474.txt
cuuggagugaugcaauggDQ489311.txt
agugaugcaauggggucaDQ489311.txt
cuucagacaauggcucauDQ489311.txt
cuuggcgauaugaaauguDQ489311.txt
ggccgagcaaugggaggcDQ489311.txt
uccgaagugaugcaauggEU076704.txt
agugaugcaauggggucaEU076704.txt
cuucagacaauggcucguEU076704.txt
ccuggcgauaugagauguEU076704.txt
ggcugagcaaugggaggcEU076704.txt
ugaaaugaaaugggguccEU360128.txt
ugaaaugaaaugggguccEU360129.txt
ugaaaugaaauggggucuEU360130.txt
ugaaaugaaaugggguccEU6